# Video Segmentation wih Pyscene-Detect

In [1]:
import pandas as pd
import cv2
import os
import math
import numpy as np
from numpy import genfromtxt
import pandas as pd
from matplotlib.pyplot import imshow
import glob
import logging
import multiprocessing as mp
import time
import io
import matplotlib.pyplot as plt
from matplotlib import rcParams


In [2]:
def save_frame(file_name, frame):
    if not os.path.exists(os.path.dirname(file_name)):
        try:
            os.makedirs(os.path.dirname(file_name))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
    cv2.imwrite(file_name, frame)


def create_scenes_files(file_path, output_path, ends_of_scenes, is_one_frame):
    infos     = file_path.rsplit('/',1)
    media_id  = infos[0]
    media_name = infos[-1].rsplit('.',1)[0]
    
    cap = cv2.VideoCapture(file_path)
    frameRate = cap.get(cv2.CAP_PROP_FPS)
    index = 1
    while(cap.isOpened()):
        frameId = cap.get(cv2.CAP_PROP_POS_FRAMES) #current frame number
        frame_timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)
        ret, frame = cap.read()
        if(ret != True):
            break
                
        output_frame_file = "{}/{}/cena-{}/frame_{}.jpg".format(output_path, media_name, str(index), str(frameId))
        
        try:
            if is_one_frame: 
#                 if index >= 53 and index <=55:
#                     print(output_frame_file)
#                     print(str((math.floor(frameId) % math.floor(frameRate))))
                if (math.floor(frameId) % math.floor(frameRate)) == 0:
                    #print("vai !!!!")
                    save_frame(output_frame_file, frame)
            else:
                save_frame(output_frame_file, frame)
        except Exception as e:
            print("Frame {} was not saved due exception. Skiping it !".format(output_frame_file))
            
        if math.floor(frameId) == ends_of_scenes[index-1]:
            index += 1
    cap.release()
    print ("Done!")
    return "{}/{}".format(output_path, media_name)

In [3]:
path_media      = "../videos/segundo_sol/segundo_sol_bloco_1_completo.mp4"
path_scenes_tmp = "scenes_tmp/"
path_scenes_csv = "{}/{}-Scenes.csv".format(path_scenes_tmp,path_media.split("/")[-1].split(".")[0])
path_output     = "output_scenes_frames"

cmd_scene_detect = "scenedetect -i {} -o {} -s metricas.csv detect-content list-scenes save-images".format(path_media, path_scenes_tmp)

In [4]:
os.system(cmd_scene_detect)

df_cenas   = pd.read_csv(path_scenes_csv, skiprows=1)
ends_list  = df_cenas["End Frame"].values

path_output_generated = create_scenes_files(path_media, path_output, ends_list, False)

Done!


In [5]:
df = pd.read_csv("scenes_tmp/6334458-360p_487k_h264-Scenes.csv", skiprows=1)

FileNotFoundError: File b'scenes_tmp/6334458-360p_487k_h264-Scenes.csv' does not exist